In [ ]:
import pandas as pd
import xgboost as xgb
import numpy as np
import sklearn
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import make_pipeline
from imblearn.combine import SMOTEENN
from imblearn.over_sampling import SMOTENC
import matplotlib.pyplot as plt
from PIL import Image
from scipy.interpolate import BSpline, make_interp_spline, interp1d
# import rpy2.robjects as robjects
# from rpy2.robjects.packages import importr
import csv
from dfply import *
from xgboost import XGBClassifier
import itertools
import os
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
import time
import pickle
import math
import matplotlib.pyplot as plt
from glob import glob

from joblib import parallel_backend
from joblib import Parallel, delayed

import importlib

import time
import requests

from joblib import Parallel, delayed
from joblib import parallel_backend
import seaborn as sns


In [ ]:
datafolder = '/home/hoyinchan/blue/Data/data2021/data2021/'
home_directory = "/home/hoyinchan/code/AKI_CDM_PY/"

In [ ]:
sites = ['MCRI', 'MCW', 'UIOWA', 'UMHC', 'UNMC', 'UofU', 'UTHSCSA', 'UTSW']
# stgs = ["stg23", "stg01"]
# fss =  ['nofs', 'rmscrbun']
stgs = ["stg01"]
fss =  ['nofs']
oversamples = ['raw']
model_types = ['catd']

In [ ]:
with open(datafolder+"tmp_crossroc.pkl", "rb") as f:
    barX = pickle.load(f)

In [ ]:
roc_table = np.round(barX[['site_m', 'site_d', 'roc']].pivot(index='site_m', columns='site_d', values='roc'),2)
roc_table = roc_table.astype(float)

In [ ]:
fig, ax = plt.subplots(figsize=(10,8.5))
sns.heatmap(roc_table, cmap='coolwarm', fmt = '', linewidths=0.01, linecolor='white')
ax.tick_params(axis='both', which='both', length=0)
ax.set(xticklabels=[])
ax.set(yticklabels=[])

In [ ]:
#sites = ['MCRI', 'MCW', 'UIOWA', 'UMHC', 'UNMC', 'UofU', 'UTHSCSA', 'UTSW']
site_m = 'UofU'
site_d = 'UofU'
# stgs = ["stg23", "stg01"]
# fss =  ['nofs', 'rmscrbun']
year=3000
stg = "stg23"
fs =  'nofs'
oversample = 'raw'
model_type ='catd'

print('Running cross_roc '+model_type+' on site '+site_m+'/'+site_d+":"+str(year)+":"+stg+":"+fs+":"+oversample, flush = True)
model = pickle.load(open('/home/hoyinchan/blue/Data/data2021/data2021/'+site_m+'/model_'+model_type+'_'+site_m+'_'+str(year)+'_'+stg+'_'+fs+'_'+oversample+'.pkl', 'rb'))

#load tables
X_train_m = pd.read_pickle('/home/hoyinchan/blue/Data/data2021/data2021/'+site_m+'/X_train_'+site_m+'_'+str(year)+'_'+stg+'_'+fs+'_'+oversample+'pos.pkl')
y_train_m = pd.read_pickle('/home/hoyinchan/blue/Data/data2021/data2021/'+site_m+'/y_train_'+site_m+'_'+str(year)+'_'+stg+'_'+fs+'_'+oversample+'pos.pkl')
X_test_m = pd.read_pickle('/home/hoyinchan/blue/Data/data2021/data2021/'+site_m+'/X_train_'+site_m+'_'+str(year)+'_'+stg+'_'+fs+'_'+oversample+'pos.pkl')
y_test_m = pd.read_pickle('/home/hoyinchan/blue/Data/data2021/data2021/'+site_m+'/y_train_'+site_m+'_'+str(year)+'_'+stg+'_'+fs+'_'+oversample+'pos.pkl')

X_train_d = pd.read_pickle('/home/hoyinchan/blue/Data/data2021/data2021/'+site_d+'/X_train_'+site_d+'_'+str(year)+'_'+stg+'_'+fs+'_'+oversample+'pos.pkl')
y_train_d = pd.read_pickle('/home/hoyinchan/blue/Data/data2021/data2021/'+site_d+'/y_train_'+site_d+'_'+str(year)+'_'+stg+'_'+fs+'_'+oversample+'pos.pkl')
X_test_d =  pd.read_pickle('/home/hoyinchan/blue/Data/data2021/data2021/'+site_d+'/X_test_' +site_d+'_'+str(year)+'_'+stg+'_'+fs+'_'+oversample+'pos.pkl')
y_test_d =  pd.read_pickle('/home/hoyinchan/blue/Data/data2021/data2021/'+site_d+'/y_test_' +site_d+'_'+str(year)+'_'+stg+'_'+fs+'_'+oversample+'pos.pkl')

common_features = [x for x in X_test_d.columns if x in X_train_m.columns]

X_train2_d = X_train_d[common_features]
X_test2_d = X_test_d[common_features]

X_train2_m = X_train_m.iloc[0:1]
X_test2_m = X_test_m.iloc[0:1]

X_train3_d = pd.concat([X_train2_m, X_train2_d]).iloc[1:]
X_test3_d = pd.concat([X_test2_m, X_test2_d]).iloc[1:]

X_train3_d.loc[:,X_train2_m.dtypes==bool] = X_train3_d.loc[:,X_train2_m.dtypes==bool].fillna(False)
X_test3_d.loc[:,X_test2_m.dtypes==bool] = X_test3_d.loc[:,X_test2_m.dtypes==bool].fillna(False)

pred = model.predict_proba(X_test3_d)
roc = roc_auc_score(y_test_d, pred[:,1])
print('Finished cross_roc '+model_type+' on site '+site_m+'/'+site_d+":"+str(year)+":"+stg+":"+fs+":"+oversample, flush = True)

In [ ]:
model.get_roc_curve(y_test_d, pred[:,1])

In [ ]:
onset = pd.read_pickle('/home/hoyinchan/blue/Data/data2021/data2021/'+'UPITT'+'/p0_onset_'+'UPITT'+'.pkl')

In [ ]:
onset

In [ ]:
xxx = pd.read_csv("plotmp.csv")
xxx = xxx.sort_values('r.sq spline',ascending=False)
xxx = xxx[~xxx['Feature'].str.contains('PX:')]
yyy =  pd.read_csv("Book1.csv")

In [ ]:
plt.plot(xxx['LONG_COMMON_NAME'], xxx['r.sq spline'], 'bo-', label='r^2')
plt.plot(xxx['LONG_COMMON_NAME'], xxx['Slope AUC'], 'ro-', label='AUC slope')
plt.axhline(y=0, color='black')
plt.xticks(rotation=90)
plt.legend()
plt.savefig("plotmp.png")

In [ ]:
xxx['index'] = [[x for x in yyy['index'] if z in x][0] if len([x for x in yyy['index'] if z in x])>0 else None for z in xxx['Feature']]

In [ ]:
xxx = xxx.merge(yyy, how='left')